In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset
import cassio


In [ ]:
!pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = 'ASTRA_DB_APPLICATION_TOKEN'
ASTRA_DB_ID = 'ASTRA_DB_ID'

OPENAI_API_KEY = 'OPENAI_API_KEY'

In [ ]:
pdfreader = PdfReader('file.pdf')

In [ ]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [ ]:
raw_text

'1 \n UNIT -3: Parameter Norm Penalties, Data set augmentation, Noise Robustness, Early \nstopping, Bagging and other Ensemble methods, Dropouts.  \nI. Parameter Norm Penalties: Parameter norm penalties, also known as regularization \ntechniques, are methods used to prevent overfitting and improve the generalization \nperformance of machine learning models. These penalties add a term to the loss \nfunction during training that encourages the model\'s weights to stay small or have a \nsparse distribution. By constraining the weights, parameter norm penalties help \nprevent the model from fitting the noise in the training data and promote more robust \nand simpler solutions. Examples of parameter norm penalties include L1 \nregularization (Lasso) and L2 regularization (Ridge regressi on). \nParameter norm penalties, such as L1 and L2 regularization, are techniques used in machine \nlearning to prevent overfitting and improve the performance of models.  \nOverfitting occurs when a model b

In [ ]:
cassio.init(token = ASTRA_DB_APPLICATION_TOKEN, database_id = ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(134523074756512) aba867ca-e7a3-4f67-bd60-3225bf900631-us-east-1.db.astra.datastax.com:29042:a8f70534-3a0e-4471-909c-1d8babbd2bae> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [ ]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

In [ ]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name = 'QA_mini_demo',
    session = None,
    keyspace = None
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:25]

["1 \n UNIT -3: Parameter Norm Penalties, Data set augmentation, Noise Robustness, Early \nstopping, Bagging and other Ensemble methods, Dropouts.  \nI. Parameter Norm Penalties: Parameter norm penalties, also known as regularization \ntechniques, are methods used to prevent overfitting and improve the generalization \nperformance of machine learning models. These penalties add a term to the loss \nfunction during training that encourages the model's weights to stay small or have a \nsparse distribution. By constraining the weights, parameter norm penalties help \nprevent the model from fitting the noise in the training data and promote more robust \nand simpler solutions. Examples of parameter norm penalties include L1 \nregularization (Lasso) and L2 regularization (Ridge regressi on).",
 "and simpler solutions. Examples of parameter norm penalties include L1 \nregularization (Lasso) and L2 regularization (Ridge regressi on). \nParameter norm penalties, such as L1 and L2 regularizatio

In [ ]:
astra_vector_store.add_texts(texts[:25])

print('Inserted %i headlines.'% len(texts[:25]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

Inserted 25 headlines.


In [ ]:
first_question = True
while True:
  if first_question:
    query_text = input('\nEnter your question (or type \'quit\' to exit)').strip()
  else:
    query_text = input('\What\'s your next question(or type \'quit\' to exit)').strip()

  if query_text.lower() == 'quit':
    break

  if query_text == '':
    continue

  first_question = False

  print('\nQUESTION: "%s"'%query_text)

  answer = astra_vector_index.query(query_text, llm = llm).strip()
  print('Answer: "%s"'%answer)

  print('FIRST DOCUMENT BY RELEVANCE:')
  for doc, score in astra_vector_store.similarity_search_with_score(query_text, k = 4):
    print('     [%0.4f] "%s  ..."'% (score, doc.page_content[:100]))



Enter your question (or type 'quit' to exit)quit
